In [5]:
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
class MeruNumberTranslator:
    def __init__(self):
        self.number_mapping = {}
        self.vectorizer = CountVectorizer()
        self.model = LogisticRegression()

    def load_dataset(self, dataset_path):
        with open(dataset_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                meru_number, equivalent, grammar = row
                self.number_mapping[meru_number] = (equivalent, grammar)

    def train(self, dataset_path):
        self.load_dataset(dataset_path)
        # Extract features and labels from the dataset
        meru_numbers = list(self.number_mapping.keys())
        number_equivalents = [entry[0] for entry in self.number_mapping.values()]

        # Convert text inputs to numerical features
        features = self.vectorizer.fit_transform(meru_numbers)

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(features, number_equivalents, test_size=0.2, random_state=42)

        # Train the model
        self.model.fit(X_train, y_train)

        # Evaluate accuracy on the testing set
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")

    def translate_to_equivalent(self, meru_number):
        features = self.vectorizer.transform([meru_number])
        equivalent, grammar = self.model.predict(features)[0], ""
        if meru_number in self.number_mapping:
            grammar = self.number_mapping[meru_number][1]
        return equivalent, grammar

    def translate_to_meru(self, number):
        for meru_number, (equivalent, grammar) in self.number_mapping.items():
            if equivalent == str(number):
                return meru_number, grammar

            return 'Unknown', 'Unknown'


translator = MeruNumberTranslator()


In [10]:
# Training of the model.
translator.train('MeruNumbers.csv')

Accuracy: 0.00


In [8]:
while True:
    print("Please select an option:")
    print("1. Translate a number to its Meru equivalent")
    print("2. Translate a Meru number to its number value equivalent")
    print("Enter 'q' to quit.")

    choice = input("Your choice: ")

    if choice == '1':
        number_input = int(input("Enter a number: "))
        meru_equivalent, grammar = translator.translate_to_meru(number_input)
        print(f"The Meru equivalent of {number_input} is: {meru_equivalent}")
        print(f"Morphological Grammar: {grammar}")
    elif choice == '2':
        meru_input = input("Enter a Meru number: ")
        equivalent, grammar = translator.translate_to_equivalent(meru_input)
        print(f"The equivalent of {meru_input} is: {equivalent}")
        print(f"Morphological Grammar: {grammar}")
    elif choice.lower() == 'q':
        break
    else:
        print("Invalid choice. Please try again.")


Please select an option:
1. Translate a number to its Meru equivalent
2. Translate a Meru number to its number value equivalent
Enter 'q' to quit.
The Meru equivalent of 1 is: Unknown
Morphological Grammar: Unknown
Please select an option:
1. Translate a number to its Meru equivalent
2. Translate a Meru number to its number value equivalent
Enter 'q' to quit.
